<a href="https://colab.research.google.com/github/angel539/Python-Notebooks/blob/main/Comparando_Clasificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ángel Mora Segura** | Científico de Datos

https://www.linkedin.com/in/angelmoras/

---

> Este notebook está inspirado en el siguiente enlace: https://towardsdatascience.com/machine-learning-classifiers-comparison-with-python-33149aecdbca

**Problema:**

> Me han dicho que haga 12 ejercicios. Tengo que aplicar 4 clasificadores y 3 métodos de validación y encima las métricas de evaluación pueden variar con el tiempo... y creo que no me da tiempo. ¿Ahora que hago?

**Solución:**

> Pues tienes dos opciones: (*1*) meterte debajo de la cama y llorar, o (*2*) **usar lo que ya sabes hacer**. Es decir, usando unos *dataframe*, unos *diccionarios* y alguna *función* para intentar hacer todo a la vez.

## 1. Importando librerias y dependencias.

**Para los dataframe:**

In [ ]:
import pandas as pd

**Para los clasificadores:**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

**Para las métricas:**

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

**Para los métodos:**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

**Para el dataset del ejemplo:**

In [ ]:
from sklearn.datasets import load_iris

## 2. Métricas de evaluación.

### 2.1 Usando el dataset de ejemplo `iris`.

Vamos a importar un dataset del conjunto de ejemplo llamado `iris` de la libreria `sklearn`.

In [ ]:
X, y = load_iris(return_X_y=True)

Una vez que tenemos el dataset cargado y los conjuntos X e y creados, vamos a dividir esos conjuntos en un subconjunto de `entrenamiento` y `prueba`:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (105, 4) (105,)
Test set: (45, 4) (45,)


### 2.2 Creando un diccionario que va a almacenar las **métricas de evaluación**.

Primero, vamos a hacer una pequeña aclaración.

> `Accuracy` es una métrica de evaluación. `Hold out`, `Leave one out` y `Kfold cross validation` son métodos de validación. No son métricas de evaluación.

Las **métricas de evaluación** se basan en el concepto de  true positive (TP). Es decir elementos que **están presentes en ambos conjuntos a comparar**. En caso de que estén presentes en  un conjunto o en otro conjunto estos elementos se denominarán FP (false positive) o FN (false negative). En caso de que el elemento no deba estar presenten en ninguno de los conjunto se trata de un TN o true negative.

> Ahora mismo conocéis la **métrica de accuracy** que es la proporción de TPs y TNs que hay presentes en los conjuntos comparados. Pero hay más y todas están interrelacionadas.

> **No debéis confundir nunca las métricas de evaluación con los métodos de validación.**

El siguiente diccionario va a contener todas las métricas de evaluación que quiero calcular, y además, va a usar la función `make_scorer()`. La función `make_scorer()` hace lo siguiente: Coge una función de validación por parámetro y devuelve su resultado. Mi diccionario se va a llamar `scoring` y se lo vamos a ir pasando a cada uno de los métodos de validación.


In [ ]:
# Definiendo un diccionario para las métricas de validación.
scoring = {
    'accuracy' : make_scorer(accuracy_score)
}

### 2.3 Construyendo los clasificadores:

Ahora, vamos a construir nuestros clasificadores. En este caso son 4. El de la `regresión logistica`, el `nearest centroid`, el `KNN` y el `Radius KNN`.

In [ ]:
# Construyendo los clasificadores.
log_model         = LogisticRegression(max_iter=10000)
nearest_centroid  = NearestCentroid()
knn_classifier    = KNeighborsClassifier(11)
radius_classifier = RadiusNeighborsClassifier(1)
# ... y si tuviera aquí más clasificadores?

### 2.4 K-fold cross validation.

Ahora, vamos a definir una función a la que vamos a llamar **metrics_Kfold_cv()** y a la que le vamos a pasar directamente nuestra `X` y nuestra `y`. Esa función, va a coger cada uno de los clasificadores y va a calcular las métricas de evaluación definidas en el diccionario de `scoring` anterior.

In [ ]:
# Define the models evaluation function
def metrics_Kfold_cv(X, y, folds):
    '''
    X : Data set con las features (selectores)
    y : Data set con el target (objetivo de la clasificación)
    folds : Número de folds en el CV.
    '''
    # Perform cross-validation to each ML classifier
    log     = cross_validate(log_model, X, y, cv=folds, scoring=scoring)
    nearest = cross_validate(nearest_centroid, X, y, cv=folds, scoring=scoring) 
    knn     = cross_validate(knn_classifier, X, y, cv=folds, scoring=scoring)
    radius  = cross_validate(radius_classifier, X, y, cv=folds, scoring=scoring)
    # ... y si tuviera aquí más clasificadores?
    
    models_scores_table = pd.DataFrame({
                                        'Logistic Regression':[
                                                  log['test_accuracy'].mean()
                                        ],
                                        'Nearest centroid':[
                                                  nearest['test_accuracy'].mean()
                                        ],
                                        'KNN Classifier':[
                                                  knn['test_accuracy'].mean()
                                        ],
                                        'Radius Classifier':[
                                                  radius['test_accuracy'].mean()
                                        ]
                                        }, index=['Accuracy'])
    
    models_scores_table['Best Score'] = models_scores_table.idxmax(axis=1)
    return(models_scores_table)

Finalmente, el siguiente DataFrame (`df_kfold`) contiene todos los resultados para el Kfold para los clasificadores seleccionados.

In [ ]:
# DataFrame con el resultado de ejecutar todos los clasificadores a la vez.
df_kfold = metrics_Kfold_cv(X, y, 5)
df_kfold.to_csv("kfold.csv")